In [1]:
import numpy as np
import torch
TRAIN_SENTENCE_PATH = "dataset/sentTrain.txt"
TRAIN_BAG_PATH = "dataset/bag_relation_train.txt"
DEV_SENTENSE_PATH = "dataset/sentDev.txt"
DEV_BAG_PATH = "dataset/bag_relation_dev.txt"
TEST_SENTENSE_PATH = "dataset/sent_test.txt"
TEST_BAG_PATH = "dataset/bag_relation_test.txt"
EMMBEDDING_LEN = 200
train_Sentense = []
train_Bag = []
dev_Sentense = []
dev_Bag = []
test_Sentense = []
test_Bag = []

In [2]:
class Sentense:
    def __init__(self, info, location):
        self.personA = info[1]
        self.personB = info[2]
        self.sentense = info[3].split(" ")
        self.sentense_emm = np.zeros((len(self.sentense),EMMBEDDING_LEN))
        if location != "test":
            self.id = int(info[4].split('_')[3])
        else:
            self.id = int(info[0].split('_')[3])            
        if location != "test":
            self.relation = []
            for label in info[5].rstrip().split(" "):
                self.relation.append(int(label))
            
class Bag:
    def __init__(self, info, location):
        self.id = info[0].split('_')[3]
        self.personA = info[1]
        self.personB = info[2]
        self.Sentenses = []
        for sen_id in info[3].split(' '):
            sen_id = int(sen_id.split("_")[3]) - 1
            if location == "train":
                self.Sentenses.append(train_Sentense[sen_id])
            elif location == "dev":
                self.Sentenses.append(dev_Sentense[sen_id])
            elif location == "test":
                self.Sentenses.append(test_Sentense[sen_id])
        if location != "test":
            self.relation = []
            for label in info[4].rstrip().split(" "):
                self.relation.append(int(label))

In [3]:
with open(TRAIN_SENTENCE_PATH) as f:
    lines = f.readlines()
    for line in lines:
        info = line.split("\t")
        tmp = Sentense(info, "train")
        train_Sentense.append(tmp)

In [4]:
with open(TRAIN_BAG_PATH) as f:
    lines = f.readlines()
    for line in lines:
        info = line.split('\t')
        tmp = Bag(info,"train")
        train_Bag.append(tmp)

In [5]:
len(train_Sentense),len(train_Bag)

(287351, 37948)

In [6]:
sen_NA = []
tmp = [0 for i in range(35)]
for sen in train_Sentense:
    for r in sen.relation:
        tmp[r]+=1
        if r == 0:
            sen_NA.append(sen)
tmp

[248850,
 8142,
 218,
 183,
 5544,
 245,
 69,
 291,
 40,
 9,
 6870,
 1383,
 2627,
 830,
 46,
 19,
 1673,
 637,
 532,
 805,
 77,
 77,
 22,
 158,
 30,
 13,
 119,
 67,
 24,
 165,
 1610,
 1301,
 1266,
 2911,
 547]

In [7]:
with open(DEV_SENTENSE_PATH) as f:
    lines = f.readlines()
    for line in lines:
        info = line.split("\t")
        tmp = Sentense(info, "train")
        dev_Sentense.append(tmp)

In [8]:
with open(DEV_BAG_PATH) as f:
    lines = f.readlines()
    for line in lines:
        info = line.split('\t')
        tmp = Bag(info,"dev")
        dev_Bag.append(tmp)

In [9]:
len(dev_Sentense),len(dev_Bag)

(38417, 5416)

In [10]:
with open(TEST_SENTENSE_PATH) as f:
    lines = f.readlines()
    for line in lines:
        info = line.split("\t")
        tmp = Sentense(info, "test")
        test_Sentense.append(tmp)

In [11]:
with open(TEST_BAG_PATH) as f:
    lines = f.readlines()
    for line in lines:
        info = line.split('\t')
        tmp = Bag(info,"test")
        test_Bag.append(tmp)

In [12]:
len(test_Sentense),len(test_Bag)

(77092, 10849)

In [13]:
# 所有切分后的句子，用于训练词向量
sentenses = []

In [14]:
for Sentense in (train_Sentense+dev_Sentense+test_Sentense):
    sentenses.append(Sentense.sentense)

In [15]:
len(sentenses)

402860

In [16]:
# from gensim.models import Word2Vec 
# model = Word2Vec(sentenses, sg=1, size=EMMBEDDING_LEN, window=5, min_count=1, negative=3)
# model.save("CCKS_2019") 

In [17]:
from gensim.models import Word2Vec 
model = Word2Vec.load("/Users/denhiroshi/Downloads/model_data/CCKS_2019")

In [18]:
MAX_LEN = 0
for Sentense in(train_Sentense+dev_Sentense+test_Sentense):
    sentense_emm = []
    for i in range(len(Sentense.sentense)):
        tmp = Sentense.sentense[i]
        sentense_emm.append(model[tmp])
    Sentense.sentense_emm = np.stack(sentense_emm)
    MAX_LEN = max(MAX_LEN, len(Sentense.sentense_emm))

/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
MAX_LEN

50

In [20]:
for Sentense in(train_Sentense+dev_Sentense+test_Sentense):
    if len(Sentense.sentense_emm) < MAX_LEN:
        need = MAX_LEN - len(Sentense.sentense_emm)
        need = np.zeros((need, EMMBEDDING_LEN))
        Sentense.sentense_emm = np.concatenate((Sentense.sentense_emm, need), axis = 0)

In [21]:
train_Sentense[0].sentense_emm.shape

(50, 200)

In [42]:
# sen_NA = []
# for Sentense in train_Sentense:
#     if 0 in Sentense.relation:
#         sen_NA.append(Sentense.sentense_emm)

# sen_NA = np.stack(sen_NA)
# np.save('/Users/denhiroshi/Downloads/model_data/train_NA.npy',sen_NA)

# 随机取样3K条无关系数据

In [22]:
sen_NA = np.load('/Users/denhiroshi/Downloads/model_data/train_NA.npy')

In [23]:
CHOOSENUMS = 5000
indexs = np.random.choice(sen_NA.shape[0], CHOOSENUMS, False)

In [24]:
sen_NA = sen_NA[indexs]
sen_NA.shape

(5000, 50, 200)

In [25]:
train_Sentense_np = []
for Sentense in train_Sentense:
    if 0 in Sentense.relation:
        continue
    train_Sentense_np.append(Sentense.sentense_emm)
train_Sentense_np = np.stack(train_Sentense_np)
train_Sentense_np = np.concatenate((train_Sentense_np, sen_NA), axis = 0)

In [26]:
dev_Sentense_np = []
for Sentense in dev_Sentense:
    dev_Sentense_np.append(Sentense.sentense_emm)
dev_Sentense_np = np.stack(dev_Sentense_np)

In [27]:
test_Sentense_np = []
for Sentense in test_Sentense:
    test_Sentense_np.append(Sentense.sentense_emm)
test_Sentense_np = np.stack(test_Sentense_np)

In [28]:
np.save('train_Sentense_np.npy',train_Sentense_np)
np.save('dev_Sentense_np.npy',dev_Sentense_np)
np.save('test_Sentense_np.npy',test_Sentense_np)

In [29]:
MAX_RELATION_LEN = 0
MU = 0 
for Sentense in(train_Sentense+dev_Sentense):
    Sentense.relation = np.stack(Sentense.relation)
    MAX_RELATION_LEN = max(MAX_RELATION_LEN, len(Sentense.relation))

In [30]:
MAX_RELATION_LEN

2

In [31]:
MU = 0 
for Sentense in(train_Sentense+dev_Sentense):
    if len(Sentense.relation) == MAX_RELATION_LEN:
        MU+=1
    else:
        need = MAX_RELATION_LEN - len(Sentense.relation)
        tmp = [-1 for i in range(need)]
        tmp = np.array(tmp)
        Sentense.relation = np.concatenate((Sentense.relation,tmp))

In [32]:
train_Sentense[0].relation

array([ 0, -1])

In [33]:
MU

49

In [36]:
train_Relation_np = []
for Sentense in train_Sentense:
    if 0 in Sentense.relation:
        continue
    train_Relation_np.append(Sentense.relation)
train_Relation_np = np.stack(train_Relation_np)
train_Relation_np = np.concatenate((train_Relation_np, np.array([[0,-1] for i in range(CHOOSENUMS)])), axis=0)


In [37]:
dev_Relation_np = []
for Sentense in dev_Sentense:
    dev_Relation_np.append(Sentense.relation)
dev_Relation_np = np.stack(dev_Relation_np)

In [38]:
np.save('train_Relation_np.npy',train_Relation_np)
np.save('dev_Relation_np.npy',dev_Relation_np)

In [39]:
with open("sentense_explain.txt",'w') as f:
    f.write("train_Sentense_np: "+str(len(train_Sentense))+'*'+str(MAX_LEN)+'*200\n')
    f.write("dev_Sentense_np: "+str(len(dev_Sentense))+'*'+str(MAX_LEN)+'*200\n')
    f.write("test_Sentense_np: "+str(len(test_Sentense))+'*'+str(MAX_LEN)+'*200\n')
    f.write("Relation属性为-1表示为填充\n")
    f.write("train_Relation_np: "+str(len(train_Sentense))+'*'+str(MAX_RELATION_LEN)+'\n')
    f.write("dev_Relation_np: "+str(len(dev_Sentense))+'*'+str(MAX_RELATION_LEN)+'\n')

In [40]:
train_Sentense_np.shape, dev_Sentense_np.shape, test_Sentense_np.shape

((43501, 50, 200), (38417, 50, 200), (77092, 50, 200))

In [41]:
train_Relation_np.shape, dev_Relation_np.shape

((43501, 2), (38417, 2))